In [5]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.playlistItems.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os
from time import sleep

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from IPython.display import clear_output

from utils import Heartbeat

scopes = [
    "https://www.googleapis.com/auth/youtubepartner",
    "https://www.googleapis.com/auth/youtube",
    "https://www.googleapis.com/auth/youtube.force-ssl",
]
reqFreq = 1.0

In [2]:
# Setup
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name    = "youtube"
api_version         = "v3"
client_secrets_file = "../keys/client_secrets.json"

def get_flow_credentials_yt( client_secrets_file, scopes ):
    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, 
        scopes
    )
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, 
        api_version, 
        credentials = credentials
    )
    clear_output( wait = False ) # Prevent keys from being stored in output
    return flow, credentials, youtube

flow, credentials, youtube = get_flow_credentials_yt( client_secrets_file, scopes )

In [3]:
from pprint import pprint
from random import shuffle

def fetch_entire_playlist( playlistId, part ):
    """ Send serial requests until info about all playlist items is obtained """
    
    limiter = Heartbeat( reqFreq )
    
    # Init
    request = youtube.playlistItems().list(
        playlistId = playlistId ,
        part       = part ,
        maxResults = 50
    )
    response = request.execute()
    nextPage = response["nextPageToken"]
    items    = response['items']
    last     = False
    
    while 1:
        request = youtube.playlistItems().list(
            playlistId = "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i",
            part       = "contentDetails,id,snippet,status",
            pageToken  = nextPage,
            maxResults = 50
        )
        response = request.execute()
        try:
            sleep( 1.0 )
            nextPage = response["nextPageToken"]
        except KeyError:
            last = True
        items.extend( response['items'] )
        if last:
            break
        else:
            limiter.rest()
        
    return items


def reorder_entire_playlist( itemList ):
    """ Given a list of video dictionaries, reorder them randomly in their parent playlist """
    N    = len( itemList )
    n    = int( N/2 ) 
    posn = list( range( N-1, -1, -1 ) )
    sccs = 0
    
    halfList = itemList[:n]
    
    shuffle( halfList )
    
    limiter = Heartbeat( reqFreq )
    
    for i, item in enumerate( halfList ):
        # Notify
        print( f'\n=== Item {i+1} of {N} ===\n' )
        # Reorder
        print( f'\tMoving {item["snippet"]["position"]} --to-> {posn[i]} ' )
        request = youtube.playlistItems().update(
            part = 'id,snippet',
            body = {
                'id': item['id'],
                'snippet': {
                    'playlistId': item['snippet']['playlistId'],
                    'resourceId': item['snippet']['resourceId'],
                    'position'  : posn[i],
                }
            }
        )
        try:
            response = request.execute()
        except HttpError:
            sleep( 1.0 )
            response = request.execute()
        # Check
        if response['snippet']['position'] == posn[i]:
            sccs += 1
            print( f'SUCCESS: {response["snippet"]["position"]}' )
        else:
            print( f'FAILURE: {response["snippet"]["position"]}, desired {posn[i]}' )
        limiter.rest()
        
    # Notify
    print( f'Success: {sccs} / {N}' )
    
    

In [6]:
ANALYZE = 0
REORDER = 1

for playlistId in [
        "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i", # Study Music 1
        "PLxgoClQQBFjilUnuo7hZNlulnTdn1f4DK", # Study Music 2
        "PLxgoClQQBFjjfRkJ-4zhVRn37rmczvTFx", # Study Music 3
        "PLxgoClQQBFjgiqSn5WkVa1slbCxTU12SI", # Study Music 4
        "PLxgoClQQBFji4MvFovpVY4MJJkaxUgiyi", # Study Music 5
        "PLxgoClQQBFjgxfyQm22P3MnbWyV74q4MR", # Study Music 6
    ][1:]:
    

    # playlistId = "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i" 
    part       = "contentDetails,id,snippet,status" 
    items = fetch_entire_playlist( playlistId, part )

    if ANALYZE:
        N      = len( items )
        repeat = 0
        unqSet = set([])

        for res in items:
            _id = res['id']
            if _id in unqSet:
                print( f"REPEAT found at {res['id']}" )
                repeat += 1
            else:
                unqSet.add( _id )

        print( f"{len(unqSet)} / {N} items are unique!" )
    
    if REORDER:
        print( '\n\n==== Reorder! ====\n' )
        reorder_entire_playlist( items )
        
        
    clear_output( wait = False )
        



==== Reorder! ====


=== Item 1 of 300 ===

	Moving 38 --to-> 299 
SUCCESS: 299

=== Item 2 of 300 ===

	Moving 99 --to-> 298 
SUCCESS: 298

=== Item 3 of 300 ===

	Moving 1 --to-> 297 
SUCCESS: 297

=== Item 4 of 300 ===

	Moving 77 --to-> 296 
SUCCESS: 296

=== Item 5 of 300 ===

	Moving 96 --to-> 295 
SUCCESS: 295

=== Item 6 of 300 ===

	Moving 59 --to-> 294 
SUCCESS: 294

=== Item 7 of 300 ===

	Moving 25 --to-> 293 
SUCCESS: 293

=== Item 8 of 300 ===

	Moving 62 --to-> 292 
SUCCESS: 292

=== Item 9 of 300 ===

	Moving 60 --to-> 291 
SUCCESS: 291

=== Item 10 of 300 ===

	Moving 27 --to-> 290 
SUCCESS: 290

=== Item 11 of 300 ===

	Moving 100 --to-> 289 
SUCCESS: 289

=== Item 12 of 300 ===

	Moving 145 --to-> 288 
SUCCESS: 288

=== Item 13 of 300 ===

	Moving 79 --to-> 287 
SUCCESS: 287

=== Item 14 of 300 ===

	Moving 103 --to-> 286 
SUCCESS: 286

=== Item 15 of 300 ===

	Moving 67 --to-> 285 
SUCCESS: 285

=== Item 16 of 300 ===

	Moving 28 --to-> 284 
SUCCESS: 284

=== Item

NameError: name 'HttpError' is not defined